In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import fa_support
from evall import *
import random

In [2]:
numbers = [ 55, 5550, 2, 2 ]
names = ['country', 'postcode', 'gender', 'loyalty']
dic = dict(zip(names,numbers))

In [3]:
item_emb_dim = sum(dic.values())

D_emb_dim = 50

G_emb_dim = 50

hidden_dim = 128
alpha = 0

# Initializer
init = tf.initializers.glorot_normal()

'''Generator and Discriminator Attribute Embeddings'''
D_country_embs = tf.keras.layers.Embedding(input_dim = dic['country'], output_dim = D_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['country'],D_emb_dim))])
D_postcode_embs = tf.keras.layers.Embedding(input_dim = dic['postcode'], output_dim = D_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['postcode'],D_emb_dim))])
D_gender_embs = tf.keras.layers.Embedding(input_dim = dic['gender'], output_dim = D_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['gender'],D_emb_dim))])
D_loyalty_embs = tf.keras.layers.Embedding(input_dim = dic['loyalty'], output_dim = D_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['loyalty'],D_emb_dim))])


G_country_embs = tf.keras.layers.Embedding(input_dim = dic['country'], output_dim = G_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['country'],G_emb_dim))])
G_postcode_embs = tf.keras.layers.Embedding(input_dim = dic['postcode'], output_dim = G_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['postcode'],G_emb_dim))])
G_gender_embs = tf.keras.layers.Embedding(input_dim = dic['gender'], output_dim = G_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['gender'],G_emb_dim))])
G_loyalty_embs = tf.keras.layers.Embedding(input_dim = dic['loyalty'], output_dim = G_emb_dim,
                                          trainable=True, weights = [init(shape=( dic['loyalty'],G_emb_dim))])


# Model input sizes
G_input_size =  G_emb_dim*4
D_input_size = item_emb_dim + D_emb_dim*4

In [4]:
def generator_input(country, postcode, gender, loyalty):
    dic = {}
    dic["country"] = G_country_embs(tf.constant(country))
    dic["postcode"] = G_postcode_embs(tf.constant(postcode))
    dic["gender"] = G_gender_embs(tf.constant(gender))
    dic["loyalty"] = G_loyalty_embs(tf.constant(loyalty))
    emb = tf.keras.layers.concatenate(list(dic.values()), 1)
    return emb

# Generates user based on concatenation of all attributes
def generator():
    bc_input = tf.keras.layers.Input(shape=(G_input_size))
    x = tf.keras.layers.Dense(hidden_dim, activation ='sigmoid', activity_regularizer = 'l2')(bc_input)
    x = tf.keras.layers.Dense(hidden_dim, activation ='sigmoid', activity_regularizer = 'l2')(x)
    x = tf.keras.layers.Dense(item_emb_dim, activation ='sigmoid', activity_regularizer = 'l2')(x)
    g_model = tf.keras.models.Model(bc_input, x, name = 'generator')
    return g_model
g_model = generator()

def discriminator_input(country, postcode, gender, loyalty, item_emb):
    dic = {}
    dic["country"] = D_country_embs(tf.constant(country))
    dic["postcode"] = D_postcode_embs(tf.constant(postcode))
    dic["gender"] = D_gender_embs(tf.constant(gender))
    dic["loyalty"] = D_loyalty_embs(tf.constant(loyalty))
    item_emb = tf.cast(item_emb, dtype=float)
    emb = tf.keras.layers.concatenate(list(dic.values()), 1)
    final_emb = tf.keras.layers.concatenate([emb, item_emb], 1)
    return final_emb

def discriminator():
    d_input = tf.keras.layers.Input(shape=(D_input_size))
    x = tf.keras.layers.Dense(hidden_dim, activation ='sigmoid', kernel_regularizer = 'l2')(d_input)
    x = tf.keras.layers.Dense(hidden_dim, activation ='sigmoid', kernel_regularizer = 'l2')(x)
    x = tf.keras.layers.Dense(1)(x)
    model = tf.keras.models.Model(d_input, x, name = 'discriminator')
    return model
d_model = discriminator()

In [5]:
train = np.load("new_fa_train_data.npy")

In [6]:
def generator_loss(fake):
    return -tf.reduce_mean(fake)

def discriminator_loss(real, fake):
    logit = tf.reduce_mean(fake- real)
    return logit

def counter_loss(counter):
    return tf.reduce_mean(counter)


In [9]:
# WGAN Class
class WGAN(tf.keras.Model):
    def __init__(
        self,
        discriminator,
        generator,
        discriminator_extra_steps=5,
        batch_size = 577
    ):
        super(WGAN, self).__init__()
        self.discriminator = d_model
        self.generator = g_model
        self.d_steps = discriminator_extra_steps
        self.batch_size = batch_size
        self.k = 10        
        self.index = 0 
        self.c_index = 0 
        self.gp_weight = 10
        self.eval_steps = 0
        self.max_p10 = .01 
        self.max_g10 = .01
        self.max_m10 = .01
        self.max_p20 = .01
        self.max_g20 = .01
        self.max_m20 = 0.01
    def compile(self, d_optimizer, g_optimizer, d_loss_fn, g_loss_fn,c_loss_fn, run_eagerly):
        super(WGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_fn = d_loss_fn
        self.c_loss_fn = c_loss_fn
        self.g_loss_fn = g_loss_fn
        self.run_eagerly = run_eagerly



    def gradient_penalty(self, batch_size, real_items, fake_items, country, postcode, gender, loyalty):
        """ Calculates the gradient penalty.

        This loss is calculated on an interpolated image
        and added to the discriminator loss.
        """
        # Get the interpolated image
        alpha = tf.random.normal([batch_size,1], 0.0, 1.0)
        diff = fake_items - real_items
        interpolated = real_items + alpha * diff

        with tf.GradientTape() as tape:
            tape.watch(interpolated)
            # 1. Get the discriminator output for this interpolated image.
            interpolated_input = discriminator_input(  country, postcode, gender, loyalty,  interpolated)
            pred = self.discriminator(interpolated_input)

        # 2. Calculate the gradients w.r.t to this interpolated image.
        grads = tape.gradient(pred, [interpolated])[0] #+1e-10
        # 3. Calculate the norm of the gradients.
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=1))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp

    def train_step(self, real_users):
        self.eval_steps +=1 
        c_batch_size = 2*self.batch_size
        for i in range(self.d_steps):

            with tf.GradientTape() as tape:
                # Get batch data
                country, postcode, loyalty, gender,  brand_id, category, colour, divisioncode, \
                itemcategorycode, itemfamilycode, itemseason, productgroup, user, real_items = fa_support.get_batchdata(self.index, self.index + self.batch_size)
                # Get batch of counter examples
                #counter_brand_id, counter_category, counter_colour, counter_divisioncode, \
                #counter_itemcategorycode, counter_itemfamilycode, counter_itemseason, \
                #counter_productgroup,  counter_users = support.get_counter_batch(self.c_index, self.c_index + c_batch_size)
                # Generate fake users from attributes
                g_input0 = generator_input(country, postcode, gender, loyalty)
                fake_items = self.generator(g_input0)
                # Get the logits for the fake users
                d_input0 = discriminator_input(country, postcode, gender, loyalty, fake_items)
                fake_logits = self.discriminator(d_input0)
                # Get the logits for the real user
                d_input1 = discriminator_input( country, postcode, gender, loyalty, real_items)
                real_logits = self.discriminator(d_input1)
                # Get logits for counter examples
                
                #d_input2 = discriminator_input(counter_brand_id, counter_category, counter_colour, counter_divisioncode, \
                #counter_itemcategorycode, counter_itemfamilycode, counter_itemseason, \
                #counter_productgroup,  counter_users)
                #counter_logits = self.discriminator(d_input2)
                # Calculate the discriminator loss using the fake and real image logits
                d_cost = self.d_loss_fn(real_logits, fake_logits)
                #c_loss = self.c_loss_fn(counter_logits)
                # Get gradient penalty
                gp = self.gradient_penalty(self.batch_size, real_items, fake_items, country, postcode, gender, loyalty)
                # Later add counter loss
                d_loss = d_cost + gp*self.gp_weight # +c_loss

            # Get the gradients w.r.t the discriminator loss
            d_gradient = tape.gradient(d_loss, self.discriminator.trainable_variables)
            # Update the weights of the discriminator using the discriminator optimizer
            self.d_optimizer.apply_gradients(zip(d_gradient, self.discriminator.trainable_variables))

        # Train the generator
        with tf.GradientTape() as tape:

            # Generate fake images using the generator
            g_input1 = generator_input( country, postcode, gender, loyalty)
            gen_items = self.generator(g_input1)
            # Get the discriminator logits for fake images
            d_input2 = discriminator_input( country, postcode, gender, loyalty, gen_items)
            gen_logits = self.discriminator(d_input2)
            # Calculate the generator loss
            g_loss = self.g_loss_fn(gen_logits)

        # Get the gradients w.r.t the generator loss
        gen_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
        # Update the weights of the generator using the generator optimizer
        self.g_optimizer.apply_gradients(
            zip(gen_gradient, self.generator.trainable_variables)
        )
        if self.eval_steps %760==0:
            p_at_10,G_at_10,M_at_10 = wgan.test_step(10)
            p_at_20,G_at_20,M_at_20 = wgan.test_step(20)
            if p_at_10 > self.max_p10:
                self.max_p10 = p_at_10
            if G_at_10 > self.max_g10:
                self.max_g10 = G_at_10
            if M_at_10 > self.max_m10:
                self.max_m10 = M_at_10
            if p_at_20 > self.max_p20:
                self.max_p10 = p_at_20
            if G_at_20 > self.max_g20:
                self.max_g10 = G_at_20
            if M_at_20 > self.max_m20:
                self.max_m10 = M_at_20
            
            return {"d_loss": d_loss, "g_loss": g_loss, "p10":p_at_10,
                           "G10":G_at_10,"M10":M_at_10, "p20": p_at_20,"G20":G_at_20,"M20":M_at_20}
        else:
            return {"d_loss": d_loss, "g_loss": g_loss}

    def test_step(self, k):
        country, postcode, loyal, gender,  brand_id, category, colour, divisioncode, \
        itemcategorycode, itemfamilycode, itemseason, productgroup, item, user = fa_support.get_testdata(0,5000)

        test_BATCH_SIZE = user.size
        g_input1 = generator_input( country, postcode, gender, loyal)
        gen_items = self.generator(g_input1)
        sim_items = fa_support.get_intersection_similar_item(gen_items, k)
        count = 0

        count = 0
        for u, item_list in zip(user, sim_items):       
            for i in item_list:
                if ui_matrix[u,i] == 1:
                    count = count + 1            
        p_at_k = round(count/(test_BATCH_SIZE * k), 4)

        RS = []
        ans = 0.0
        for u, item_list in zip(user, sim_items):           
            r=[]
            for i in item_list:
                 r.append(ui_matrix[u][i])
            ans = ans + evall.ndcg_at_k(r, k, method=1)
            RS.append(r)
        G_at_k = ans/test_BATCH_SIZE
        M_at_k = evall.mean_average_precision(RS)


        return p_at_k,G_at_k,M_at_k

In [10]:
# Fit 
epochs = 10
# Instantiate the WGAN model.
wgan = WGAN(
    discriminator=discriminator,
    generator=generator,
    discriminator_extra_steps=3
)

# Compile the WGAN model.
wgan.compile(
    d_optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    g_optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    g_loss_fn=generator_loss,
    d_loss_fn=discriminator_loss,
    c_loss_fn = counter_loss,
    run_eagerly=True)

# Start training the model.
fit = wgan.fit(train, batch_size=100, epochs=epochs, verbose=True)
print(wgan.test_step(10), "\n", wgan.test_step(20))

Epoch 1/10
759/762 [============================>.] - ETA: 1s - d_loss: -5.7893 - g_loss: -21.4598

MemoryError: Unable to allocate 3.61 GiB for an array with shape (5609, 86317) and data type float64

In [ ]:
eval_dic["User_Model_WGAN_initial"] = [wgan.max_g10, wgan.max_g20, wgan.max_m10, wgan.max_m20, wgan.max_p10, wgan.max_p20]

In [ ]:
eval_dic